# Emoji generation using LSTM 

<i>This is the same model as "emoji_gen_lstm_V2" but instead of using LSTM layer directlty in the model function we have defined the layers objects as global and used it accordingly. Have a look:</i>

<b><i>Importing python libraries</i></b>

In [2]:
import numpy as np
from emo_utils import *
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Reshape, Lambda
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
np.random.seed(1)

Using TensorFlow backend.


<b><i>Reading dataset in train and test varibles</i></b>

In [3]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

<b><i>Maximum length of the longest sentence in the dataset</i></b>

In [4]:
maxLen = len(max(X_train, key=len).split())

<b><i>Converting the train and test label to their one hot vector form matrix</i></b>

In [6]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)
print(Y_oh_train.shape)

(132, 5)


<b><i>Function to read and store the glove embedding matrix</i></b>

In [7]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [9]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

<b><i>Function convert every sentence to its corresponding indices vector using word_to_index dictionary; it also pads zero if incase the sentence vector is less than the max length of the sentence </i></b>

In [10]:
def sentences_to_indices(X, word_to_index, max_len):
 
    m = X.shape[0]                                   
    X_indices = np.zeros((m, max_len))
    for i in range(m):                               
        
        sentence_words = X[i].lower().split()
        j = 0
        
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w] 
            j = j + 1
    
    return X_indices

<b><i>Converting our train data to trainable form</i></b>

In [11]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
X_train_indices.shape

(132, 10)

<b><i>Function builds the <u>Embedding()</u> layer in Keras. After this layer is built, we can pass the output of <u>sentences_to_indices()</u> to it as an input, and the Embedding() layer will return the word embeddings for that sentence.</i></b>

In [12]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):

    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

<b><i>Initializing parameters used in layers objects</i></b>

In [15]:
n_a = 128
n_values = 5

<b><i>Defined the layers objects we need, as global variables</i></b>

In [14]:
reshapor = Reshape((1, 50))                        
LSTM_cell = LSTM(n_a, return_state = True)         
densor = Dense(n_values, activation='softmax')

<b><i>Model function</i></b>

In [16]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index, n_a):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    X = embeddings
    print("Shape of embeddings: ", embeddings.shape)
    n_value = embeddings.shape[2]
    Tx = embeddings.shape[1]
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    for t in range(Tx-1):
        
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
    
    x = Lambda(lambda x: X[:,t,:])(X)
    x = reshapor(x)
    a, _, c = LSTM_cell(x, initial_state=[a, c])
    out = densor(a)
    
    model = Model(inputs = [sentence_indices, a0, c0], outputs = out)
    
    return model

<b><i>Calling the Model Function</i></b>

In [17]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index, n_a)

Shape of embeddings:  (?, 10, 50)


<b><i>Compiling the model</i></b>

In [18]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

<b><i>Assigning the values to be passed into the model for training</i></b>

In [19]:
m = X_train.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

<b><i>Fitting our training data to the compiled model</i></b>

In [20]:
model.fit([X_train_indices, a0, c0], Y_train_oh, epochs=100)

Epoch 1/100
132/132 [==============================] - 12s 93ms/step - loss: 1.6059 - acc: 0.2727
Epoch 2/100
132/132 [==============================] - 0s 2ms/step - loss: 1.3175 - acc: 0.5303
Epoch 3/100
132/132 [==============================] - 0s 2ms/step - loss: 1.0387 - acc: 0.6970
Epoch 4/100
132/132 [==============================] - 0s 2ms/step - loss: 0.8233 - acc: 0.6667
Epoch 5/100
132/132 [==============================] - 0s 2ms/step - loss: 0.6276 - acc: 0.7500
Epoch 6/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5297 - acc: 0.8182
Epoch 7/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4672 - acc: 0.8106
Epoch 8/100
132/132 [==============================] - 0s 1ms/step - loss: 0.4568 - acc: 0.8258
Epoch 9/100
132/132 [==============================] - 0s 1ms/step - loss: 0.2726 - acc: 0.9091
Epoch 10/100
132/132 [==============================] - ETA: 0s - loss: 0.4558 - acc: 0.875 - 0s 1ms/step - loss: 0.5368 - acc: 0.8182

132/132 [==============================] - 0s 561us/step - loss: 6.5205e-04 - acc: 1.0000
Epoch 83/100
132/132 [==============================] - 0s 569us/step - loss: 6.3728e-04 - acc: 1.0000
Epoch 84/100
132/132 [==============================] - 0s 516us/step - loss: 6.2377e-04 - acc: 1.0000
Epoch 85/100
132/132 [==============================] - 0s 622us/step - loss: 6.1263e-04 - acc: 1.0000
Epoch 86/100
132/132 [==============================] - 0s 584us/step - loss: 5.9931e-04 - acc: 1.0000
Epoch 87/100
132/132 [==============================] - 0s 576us/step - loss: 5.8822e-04 - acc: 1.0000
Epoch 88/100
132/132 [==============================] - 0s 576us/step - loss: 5.7747e-04 - acc: 1.0000
Epoch 89/100
132/132 [==============================] - 0s 569us/step - loss: 5.6717e-04 - acc: 1.0000
Epoch 90/100
132/132 [==============================] - 0s 569us/step - loss: 5.5833e-04 - acc: 1.0000
Epoch 91/100
132/132 [==============================] - 0s 576us/step - loss: 5.4953e-

<b><i>Inference_model for predicting the output from the trained lstm_cell. Here, LSTM_cell and densor layers passed as parameter are the trained cell</i></b>

In [21]:
def emoji_inference_model(LSTM_cell, densor, input_shape, word_to_vec_map, word_to_index, n_a):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    X = embeddings
    print("Shape of embeddings: ", embeddings.shape)
    n_value = embeddings.shape[2]
    Tx = embeddings.shape[1]
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    for t in range(Tx-1):
        
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
    
    x = Lambda(lambda x: X[:,t,:])(X)
    x = reshapor(x)
    a, _, c = LSTM_cell(x, initial_state=[a, c])
    out = densor(a)
    
    model = Model(inputs = [sentence_indices, a0, c0], outputs = out)
    
    return model

<b><i>Calling the inference model</i></b>

In [22]:
inference_model = emoji_inference_model(LSTM_cell, densor, (maxLen,), word_to_vec_map, word_to_index, n_a)

Shape of embeddings:  (?, 10, 50)


<b><i>Converting our test data to testable form</i></b>

In [23]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)

<b><i>Assigning and initializing the values to be passed into the inference_model for testing</i></b>

In [24]:
m = X_test.shape[0]
a_initializer = np.zeros((m, n_a))
c_initializer = np.zeros((m, n_a))

<b><i>Simple predictor function that predict the text's emoji using inference model and evaluates he accuracy from original model for training. </i></b>

In [25]:
def predict_and_evaluate(inference_model, x_initializer, a_initializer, c_initializer, Y_test_oh):

    eval_ = model.evaluate([X_test_indices, a_initializer, c_initializer], Y_test_oh)
    
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    
    for i in range(len(X_test)):
        num = np.argmax(pred[i])
        if(num != Y_test[i]):
            print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

    return eval_[0], eval_[1]

<b><i>Here's the accuracy. happy!</i></b>

In [26]:
loss, acc = predict_and_evaluate(inference_model, X_test_indices, a_initializer, c_initializer, Y_test_oh)
print("Accuracy: ", acc)

56/56 [==============================] - 2s 35ms/step
Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😄 prediction: We had such a lovely dinner tonight	🍴
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: My life is so boring	😄
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:⚾ prediction: he can pitch really well	😄
Expected emoji:❤️ prediction: family is all I have	😞
Accuracy:  0.8214285629136222
